In [ ]:
# #!sudo hostname -s 127.0.0.1

# import findspark
# findspark.init('/usr/local/Cellar/apache-spark/2.4.0/libexec')

# import pyspark
# from pyspark.sql import SQLContext

# import pandas as pd

# sc = pyspark.SparkContext(appName="testApp").getOrCreate()
# sqlContext = SQLContext(sc)

In [ ]:
# #Train spark.ml.lr model
# # https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa

# from pyspark.ml.classification import LogisticRegression
# from pyspark.sql.types import *

# d = [{'name': 'A1', 'features': 60, 'lbl':1},
#      {'name': 'B1', 'features': 55, 'lbl':1},
#      {'name': 'A1', 'features': 30, 'lbl':0},
#      {'name': 'B1', 'features': 20, 'lbl':0},
#      {'name': 'A1', 'features': 10, 'lbl':0},
#      {'name': 'B1', 'features': 30, 'lbl':0},
#      {'name': 'A1', 'features': 25, 'lbl':0},
#      {'name': 'B1', 'features': 49, 'lbl':0}]
# schema = StructType([
#     StructField("name", StringType(), True),
#     StructField("features", IntegerType(), True),
#     StructField("lbl", IntegerType(), True)
#     ])
# training = sqlContext.createDataFrame(sc.parallelize(d), schema)
# training = training.select(training['features'], training['lbl'])
# # training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
# lrModel = lr.fit(training)
# print("Coefficients: " + str(lrModel.coefficients))
# print("Intercept: " + str(lrModel.intercept))

# # mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")
# # mlrModel = mlr.fit(training)
# # print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
# # print("Multinomial intercepts: " + str(mlrModel.interceptVector))

In [ ]:
# #Choose the right threshold for spark.ml.lr model

# trainingSummary = lrModel.summary

# # Obtain the objective per iteration
# objectiveHistory = trainingSummary.objectiveHistory
# print("objectiveHistory:")
# for objective in objectiveHistory:
#     print(objective)

# # Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
# trainingSummary.roc.show()
# print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# # Set the model threshold to maximize F-Measure
# fMeasure = trainingSummary.fMeasureByThreshold
# maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
# bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
#     .select('threshold').head()['threshold']
# lr.setThreshold(bestThreshold)

In [ ]:
#Create pandas train/test df

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# all_col = ['features']
# lbl_col = 'lbl'

df = pd.DataFrame(np.random.rand(3,3))
df.columns = ['n1','n2','n3']
df['c1'] = ['cat','bat','go']
df['c2'] = ['cats','bats','gos']
df['label'] = [1,1,0]

#replicate df and add noise to df
df = pd.concat([df]*50, ignore_index=True)
noise = np.random.normal(0, 1, (df.shape[0],3))
df[['n1','n2','n3']] = df[['n1','n2','n3']] + noise

#Assign some dummy train/test variable names
all_col = ['n1','n2','n3']
# g_train, g_y_train, X_test, y_test = df[all_col].values, df[['label']].values, df[all_col].values, df[['label']].values
g_train, g_y_train, X_test, y_test = df[all_col], df[['label']], df[all_col], df[['label']]

X_train, X_val, y_train, y_val = train_test_split(g_train, g_y_train, train_size=.8, random_state=0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

df.head(5)


In [ ]:
import lightgbm as lgb
import logging
import time
import sys
import json

params = {}
x, y = X_train, y_train
validate = True

log = logging.getLogger()
log.setLevel(logging.INFO)
# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
# Set STDERR handler as the only handler 
log.handlers = [handler]

local_params = {
    'num_leaves': int(params.get('clf.num_leaves', '4096')),
    'max_depth': int(params.get('clf.max_depth', '20')),
    'num_iterations': int(params.get('clf.num_iterations', '200')),
    'metric': ['l1', 'l2'],
    'learning_rate': 0.1,
    'min_sum_hessian_in_leaf': 100,
    'min_data_in_leaf': 1,
    'is_unbalance': 'true',
    #'zero_as_missing': 'true' #There is slight increase in recall(0.2%) leaving null as is with l1/l2 metric
    'verbose_eval': int(params.get('clf.verbose', '0'))
}

time_zero = time.time()
log.info('train')
evals_result = {}
lgb_train = lgb.Dataset(x, y)
clf = lgb.train(local_params,
                  lgb_train,
                  valid_sets=[],
                  evals_result=evals_result)
log.info('training done')
time.sleep(2)
duration = '%.4f min'%((time.time() - time_zero)/60.0)

model_metrics = {'model_type':'lgbm', 'train_time':duration, 'model_params':json.dumps(local_params)}
if validate:
    #Inspect insample predictions spread on training dataset
    y_train_prediction = clf.predict(x)
    insample_prediction_spread = pd.DataFrame(y_train_prediction).describe()[0].to_dict()
    for k in insample_prediction_spread:
        val = '%.4f'%(insample_prediction_spread[k])
        key = 'v_isps_' + str(k) #v_isps -> validation insample prediction spread
        model_metrics[key] = val
log.info(model_metrics)

# ##Inspect insample corr values
# #import scipy #scipy.stats.spearmanr
# y_train_prediction = clf.predict(X_train)
# np.corrcoef(y_train_prediction, np.array(y_train['label']))


In [ ]:
# https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html
# https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTENC.html

from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier

# sampling_ratio, k_neighbors = 0.1, 5
# X_train_smote, y_train_smote = SMOTE(sampling_strategy=sampling_ratio, k_neighbors = k_neighbors, n_jobs = -1).fit_resample(X_train, y_train)
X_train_smote, y_train_smote = X_train, y_train

brf = BalancedRandomForestClassifier(n_estimators=150, random_state=0, n_jobs=-1, verbose=1)
brf.fit(X_train_smote, y_train_smote)

y_pred_brf = brf.predict(df[all_col])


In [ ]:
def get_aucpr(ground_truth, predictions):
    precision, recall, thresholds = sklearn.metrics.precision_recall_curve(ground_truth, predictions)
    area = sklearn.metrics.auc(recall, precision) #Is this same as AP ?
    return area
    #print ('AP', sklearn.metrics.average_precision_score(y_test, y_pred, average='macro'))

def get_aucroc(m, train, test, y_train, y_test): 
    train_auc, test_auc = sklearn.metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]), sklearn.metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]) 
    return test_auc

def get_training_validation_datasets_from_training_dataset(cur_df, tr_percentage=0.9):
    atr_samples = cur_df.shape[0]
    tr_st_idx, tr_end_idx = 0, int(tr_samples*tr_percentage)
    va_st_idx, va_end_idx = tr_end_idx+1, tr_samples-1
    tmp_training_df, tmp_validation_df = cur_df.iloc[tr_st_idx:tr_end_idx+1], cur_df.iloc[va_st_idx:va_end_idx+1]
    return tmp_training_df, tmp_validation_df

# tmp_training_df, tmp_validation_df = get_training_validation_datasets_from_training_dataset(training_df)
    

In [ ]:
##Hyperopt based parameter tuned version of model
# https://www.kaggle.com/dreeux/hyperparameter-tuning-using-hyperopt
# https://mlwhiz.com/blog/2017/12/28/hyperopt_tuning_ml_model/
# https://medium.com/district-data-labs/parameter-tuning-with-hyperopt-faa86acdfdce

from hyperopt import hp, tpe, Trials, STATUS_OK
from hyperopt.fmin import fmin
import xgboost as xgb

def objective(space):
    clf = xgb.XGBClassifier(objective='binary:logistic',
                            n_estimators = space['n_estimators'],
                            max_depth = space['max_depth'],
                            learning_rate = space['learning_rate'],
                            n_jobs=-1
                           )
    eval_set  = [(train, y_train), (valid, y_valid)]
    clf.fit(train,
            y_train,
            eval_set=eval_set,
            eval_metric = 'aucpr', early_stopping_rounds=5)
    pred = clf.predict_proba(valid)[:,1]
    aucpr = get_aucpr(y_valid.values, pred)
    print ("SCORE:", aucpr)
    return{'loss':aucpr, 'status':STATUS_OK}

space = {
        'max_depth': hp.choice('max_depth', np.arange(5, 25, dtype=int)),
        'n_estimators' : hp.choice('n_estimators', np.arange(100, 500, 100, dtype=int)),
        'learning_rate' : hp.quniform('learning_rate', 0.025, 0.5, 0.025),
        'scale_pos_weight' : hp.choice('scale_pos_weight',[1,2,5,10,50,100,1000])
    }

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=1,
            trials=trials)

trials_file = boosting_dataset_dir+'hyperopt_trials.bin'
pickle.dump(trials, open(trials_file, "wb"))
# trials = pickle.load(open(trails_file, "rb"))

print(best)


In [ ]:
# -conda install py-xgboost
# -conda install py-xgboost-gpu
# -pip install xgboost

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

## Parameter Tuning
# model = xgb.XGBClassifier()
# param_dist = {"max_depth": [20,50],
#               "min_child_weight" : [3,6],
#               "n_estimators": [10,20],
#               "learning_rate": [0.1,0.16],}
# grid_search = GridSearchCV(model, param_grid=param_dist, cv = 2, verbose=10, n_jobs=-1)
# grid_search.fit(train, y_train)
# print(grid_search.best_estimator_)

## https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst
max_depth, min_child_weight, n_estimators, learning_r = 6, 1, 200, 0.2
model = xgb.XGBClassifier(objective='binary:logistic', max_depth=max_depth, learning_rate=learning_r, min_child_weight=min_child_weight, n_estimators=n_estimators, n_jobs=-1, verbose=2) 

# dtrain = xgb.DMatrix(train, label=y_train)
model.fit(X_train,y_train.ravel(),eval_metric=['aucpr'],eval_set=[(X_val,y_val.ravel())],early_stopping_rounds=8)

# ## Save model to file
# file_path = boosting_dataset_dir+"maxdepth_{}_minchildweight_{}_estimators_{}_lr_{}.joblib".format(max_depth,min_child_weight,n_estimators,learning_r) 
# joblib.dump(model, file_path)
# ## Load model
# # loaded_model = joblib.load(file_path)

print(auc(model, X_train, X_test))
predicted_probabilities = pd.Series(model.predict_proba(X_test)[:,1]).reset_index(drop=True)
print(predicted_probabilities.head(5))


In [ ]:
##Feature Importance

xgb.plot_importance(model)

# xgb.plot_tree(model, num_trees=2)
# xgb.to_graphviz(model, num_trees=2)

# model.feature_names = xgtrain.feature_names
tree_pdf_dir = '/Users/maheshgoud/Documents/Data/fraud_data/Results/trees'
for tree_index in range(1,2):
    dot = xgb.to_graphviz(model, num_trees=tree_index)
    dot.render("{}/tree{}".format(tree_pdf_dir,tree_index))

In [ ]:
# https://github.com/slundberg/shap
import shap
# shap.initjs()

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
# shap.force_plot(explainer.expected_value, shap_values[0,:], train[0,:])
shap.force_plot(explainer.expected_value, shap_values, X_train)

In [ ]:
shap.summary_plot(shap_values, X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")
shap.dependence_plot("Feature 0", shap_values, X_train)